# NLP Project Demo 

**Title: Multi-task learning for Text-based Emotion Detection across disparate label spaces**

### What's in this notebook
This notebook loads a **pretrained** Multi-head multi-task learning network and performs forward pass on new testing points.

We split the qualitative demonstration into 3 sections: 
1. One pass through the entire testing set to display metrics
2. Randomly sample 3 examples from each set to show the prediction & ground truth
3. A playground where you can add your own sentences to see what the network predicts

The test set from each of the following datasets are loaded:
* Fairy Tale (Ekman labelling scheme)
* EmoBank (VAD labelling scheme)
* SemEval2018 (Non-standard labels)

For more details on the dataset we refer to the submitted paper

### How to run this notebook
This notebook is completely self-contained and runnable in a google colab environment, please run from top to bottom without skipping cells.

Ideally you should turn on GPU in google colab to run forward pass over the entire testing set. The notebook still functions with CPU but it will be much slower. 


## 0 - Install & import

In [1]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 15.2 MB/s eta 0:00:00


In [2]:
import torch 
import torch.nn as nn
import transformers
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torch import ones_like, zeros_like

import os
import math
import copy
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm
from IPython.display import display, HTML
from textwrap import wrap



from transformers import AutoModel, BertTokenizerFast

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(DEVICE)

# set manual seed 
np.random.seed(42)
torch.manual_seed(42)

cuda:0


In [4]:
!git clone https://github.com/LeonY117/EmotionAnalysis.git -q

In [5]:
# load the cleaned up dataset from github
CLEAN_DATA_DIR = "/content/EmotionAnalysis/data/clean/"
CHILDREN_filename = "children_test.csv"
EMOBANK_filename = "emobank_test.csv"
SEM_filename = "SemEval2018_test.csv"

df_children = pd.read_csv(os.path.join(CLEAN_DATA_DIR, CHILDREN_filename))
df_emobank = pd.read_csv(os.path.join(CLEAN_DATA_DIR, EMOBANK_filename))
df_sem = pd.read_csv(os.path.join(CLEAN_DATA_DIR, SEM_filename))

print(f'Fairy Tale: {len(df_children)}, EmoBank: {len(df_emobank)}, SemEval: {len(df_sem)}')

Fairy Tale: 122, EmoBank: 982, SemEval: 3259


### Define some global variables

Throughout the notebook, capitalized variables are used as global variables and sometimes called from within functions & classes. Here we define the emotions for each category.

In [6]:
EKMAN_EMOTIONS = ['anger-disgust', 'fear', 'happy', 'sad', 'surprise']
SEM_EMOTIONS = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
VAD_EMOTIONS = ['V', 'A', 'D']

# outputs heads (prediction heads)
NUM_CLASSES_EKMAN = len(EKMAN_EMOTIONS) # 5
NUM_CLASSES_SEM = len(SEM_EMOTIONS) # 11
NUM_CLASSES_VAD = len(VAD_EMOTIONS) # 3

OUT_DIMS = {
    'ekman': NUM_CLASSES_EKMAN, 'vad': NUM_CLASSES_VAD, 'sem': NUM_CLASSES_SEM
}

# label lengths (this is how many slots it takes to store the labels)
Y_DIM_EKMAN = 1
Y_DIM_VAD = NUM_CLASSES_VAD
Y_DIM_SEM = NUM_CLASSES_SEM

Y_DIMS = {
    'ekman': Y_DIM_EKMAN, 'vad': Y_DIM_VAD, 'sem': Y_DIM_SEM
}

### Process dataframe into tensors

In [7]:
x_ekman = list(df_children['sentence'])
y_ekman = df_children['label']

x_vad = list(df_emobank['text'])
y_vad = df_emobank[VAD_EMOTIONS].to_numpy()

x_sem = list(df_sem['text'])
y_sem = df_sem[SEM_EMOTIONS].to_numpy()

### Generate Task Labels

These task labels are one-hot tensors assigned for each datapoint, for example for ekman task we would have the tensor `[1, 0, 0]` associated with it. Note that these flags can be all turned on at inference time, (i.e. `[1, 1, 1]`) to indicate that we want to see all predictions for all the tasks.

In [8]:
# generate task labels
task_ekman = torch.tensor([1, 0, 0]).unsqueeze(0).repeat((len(x_ekman), 1))
task_vad = torch.tensor([0, 1, 0]).unsqueeze(0).repeat((len(x_vad), 1))
task_sem = torch.tensor([0, 0, 1]).unsqueeze(0).repeat((len(x_sem), 1))

### preprocess y
We want all of the labels to have the same length, and we pad with 0s. 

For example, a ground truth label for a VAD datapoint of `[3.0, 3.1, 3.2]` would be transformed into `[zeros, 3.0, 3.1, 3.2, zeros]`

In [9]:
# create placeholder tensors
ekman_zeros = torch.zeros((1, Y_DIMS['ekman']), )
vad_zeros = torch.zeros((1, Y_DIMS['vad']), )
sem_zeros = torch.zeros((1, Y_DIMS['sem']), )

# EKMAN
y = torch.tensor(y_ekman, dtype=torch.float).unsqueeze(-1)
n = y.shape[0]
y_ekman = torch.concatenate((y, vad_zeros.repeat(n, 1), sem_zeros.repeat(n, 1)), dim=-1)

# VAD
y = torch.tensor(y_vad, dtype=torch.float)
n = y.shape[0]
y_vad = torch.concatenate((ekman_zeros.repeat(n, 1), y, sem_zeros.repeat(n, 1)), dim=-1)

# sem
y = torch.tensor(y_sem, dtype=torch.float)
n = y.shape[0]
y_sem = torch.concatenate((ekman_zeros.repeat(n, 1), vad_zeros.repeat(n, 1), y), dim=-1)

In [10]:
# move everything to GPU
task_ekman = task_ekman.to(DEVICE)
task_vad = task_vad.to(DEVICE)
task_sem = task_sem.to(DEVICE)

y_ekman = y_ekman.to(DEVICE)
y_vad = y_vad.to(DEVICE)
y_sem = y_sem.to(DEVICE)

### Create dataset and dataloader

The dictionaries `datasets` and `dataloaders` hold the corresponding data and dataloader for `ekman`, `vad`, `sem`, and `all` (which combines all three datasets). 

In [11]:
class Emotion_dataset(Dataset):
  def __init__(self, X, y, task):
    self.X = X
    self.y = y 
    self.task = task 

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    sample = (self.X[idx], self.y[idx], self.task[idx])
    return sample

In [12]:
x_all = x_ekman + x_vad + x_sem
y_all = torch.concatenate((y_ekman, y_vad, y_sem), dim=0)
task_all = torch.concatenate((task_ekman, task_vad, task_sem), dim=0)

datasets = {}
datasets['ekman'] = Emotion_dataset(x_ekman, y_ekman, task_ekman)
datasets['vad'] = Emotion_dataset(x_vad, y_vad, task_vad)
datasets['sem'] = Emotion_dataset(x_sem, y_sem, task_sem)
datasets['all'] = Emotion_dataset(x_all, y_all, task_all)

dataloaders = {}
dataloaders['ekman'] = DataLoader(datasets['ekman'], batch_size = 16, shuffle=True)
dataloaders['vad'] = DataLoader(datasets['vad'], batch_size = 16, shuffle=True)
dataloaders['sem'] = DataLoader(datasets['sem'], batch_size = 16, shuffle=True)
dataloaders['all'] = DataLoader(datasets['all'], batch_size = 16, shuffle=True)

## 1 - Model Definition

### Download tokenizer & Bert

In [16]:
# Load the BERT tokenizer
pretrained_checkpoint = 'bert-base-uncased' 

TOKENIZER = BertTokenizerFast.from_pretrained(pretrained_checkpoint)

# import BERT-base pretrained model
BERT = AutoModel.from_pretrained(pretrained_checkpoint)

BERT.to(DEVICE)

# Freeze bert and move it to GPU
for param in BERT.parameters():
  param.requires_grad = False
BERT.to(DEVICE)
print(f'moved bert to {DEVICE}')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


moved bert to cuda:0


### Multihead 

This is the class which contains the shared base and the predictors. In training this is the only part of the network where weights get updated.

In [13]:
class MultiheadNetwork(nn.Module):
  def __init__(self, h_size=256, dropout=0):
    super().__init__()
    
    self.shared_base = nn.Linear(768, h_size)
    self.ekman_predictor = nn.Linear(h_size, OUT_DIMS['ekman'])
    self.vad_predictor = nn.Linear(h_size, OUT_DIMS['vad'])
    self.sem_predictor = nn.Linear(h_size, OUT_DIMS['sem'])

    self.dropout = nn.Dropout(p=dropout, inplace=False)
    self.relu = nn.ReLU()
    # self.softmax = nn.Softmax(dim=1)
    self.sigmoid = nn.Sigmoid()
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, X, task):  
    
    X = self.relu(self.shared_base(X))
    X = self.dropout(X)

    ekman_filter = task[:, 0].unsqueeze(-1)
    y_ekman = ekman_filter * self.ekman_predictor(X)
    y_ekman = self.softmax(y_ekman)

    vad_filter = task[:, 1].unsqueeze(-1)
    y_vad = vad_filter * self.vad_predictor(X)
    y_vad = self.relu(y_vad)

    sem_filter = task[:, 2].unsqueeze(-1)
    y_sem = sem_filter * self.sem_predictor(X)
    y_sem = self.sigmoid(y_sem)

    y = torch.concat((y_ekman, y_vad, y_sem), dim=1)

    return y

### Shared Embedding Network

In [34]:
# shared embedding network
class LELNetwork(nn.Module):
  def __init__(self, h_size=256, dropout=0):
    super().__init__()
    
    self.shared_base = nn.Linear(768, h_size)
    self.LEL = nn.Linear(h_size, OUT_DIMS['ekman']+OUT_DIMS['vad']+OUT_DIMS['sem'], bias=False)

    self.dropout = nn.Dropout(p=dropout, inplace=False)
    self.relu = nn.ReLU()
    self.softmax = nn.LogSoftmax(dim=1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, x, task):  
    x = self.relu(self.shared_base(x))
    x = self.dropout(x)
    y = self.LEL(x)
    ekman_filter = task[:, 0].unsqueeze(-1)
    vad_filter = task[:, 1].unsqueeze(-1)
    sem_filter = task[:, 2].unsqueeze(-1)

    y[:, 0:5] = self.softmax(y[:, 0:5] * ekman_filter)
    y[:, 5:8] = self.relu(y[:, 5:8] * vad_filter)
    y[:,8:] = self.sigmoid(y[:, 8:] * sem_filter)

    return y

### Full model

This model takes care of the entire pipeline from tokenization to preprocessing to prediction. 

In [14]:
class MTL_network(nn.Module):
  def __init__(self, predictor):
    super().__init__()
    self.tokenizer = TOKENIZER # should be a global variable
    self.bert = BERT # should be a global variable
    self.predictor = predictor
  
  def forward(self, sent, task):
    '''
    Args:
    -----
    sent: (n, ) array of sentences
    task: (n, 3) tensor of binary flags indicating if each task is on / off
    '''
    # tokenize sentence
    encoded_input = self.tokenizer(sent, padding=True, truncation=True, return_tensors="pt")

    # extract tokenized data and move to device
    X_input = encoded_input['input_ids'].to(DEVICE)
    X_mask = encoded_input['attention_mask'].to(DEVICE)

    # bert forward pass
    feature = self.bert(X_input, attention_mask=X_mask)['pooler_output']

    out = self.predictor(feature, task)
    
    return out

## 2- Load pretrained predictor

Here we are loading a trained multi-head network (or shared embedding network, your choice). We load the predictor part of the network and pass it as an input to the full model.

In [42]:
use_multihead = True # toggle this to switch between Multihead and shared embedding
MODEL_FOLDER = '/content/EmotionAnalysis/saved_models/'

if use_multihead:
  MODEL_NAME = 'MHMTL.pt'
  # initiate network
  predictor_net = MultiheadNetwork()
  # load checkpoint
  ckpt = torch.load(os.path.join(MODEL_FOLDER, MODEL_NAME), map_location=DEVICE)
  predictor_net.load_state_dict(ckpt)
else:
  MODEL_NAME = 'SEMTL.pt'
  # initiate network
  predictor_net = LELNetwork()
  # load checkpoint
  ckpt = torch.load(os.path.join(MODEL_FOLDER, MODEL_NAME), map_location=DEVICE)
  predictor_net.load_state_dict(ckpt)

# move to GPU
predictor_net.to(DEVICE)

net = MTL_network(predictor_net)

## 3- Metrics

Here we define the metrics for each task. 
* Fairy tales (`ekman`) uses F1 score, see `compute_F1()`
* EmoBank (`VAD`) uses Pearson's correlation coefficient, see `compute_corr()`.
* SemEval (`sem`) uses Jaccard's accuracy, see `compute_Jaccard()`

The class `MultiTaskMetric` handles metric evaluation over a batch by calling the functions above for each task. 

In [43]:
def compute_F1(y_pred, y_gt, mask, detailed=False):
  '''
  Args
  -----
  y_pred: (n x 5)
  y_gt: (n x 1)
  mask: (n x 1)
  detailed: if True, return F1 for every class

  Returns
  -----
  Jaccard_accuracy: float
  '''
  gt_class = y_gt.to(int).squeeze(dim=1)
  pred_class = torch.argmax(y_pred, dim=-1).to(int)
  mask = mask.squeeze(dim=1).to(int)

  F1s = []
  TPs, FPs, FNs = 0, 0, 0
  for c in range(NUM_CLASSES_EKMAN):
    TP = ((gt_class==c)&(pred_class==c)&(mask==1)).sum()
    FP = ((pred_class==c)&(gt_class!=c)&(mask==1)).sum()
    FN = ((pred_class!=c)&(gt_class==c)&(mask==1)).sum()
    if detailed:
      F1s.append(TP/ (TP + 0.5 * (FP + FN) + 1e-16))
    TPs += TP
    FPs += FP
    FNs += FN
  
  # print(f'TP: {TPs}, FP: {FPs}, FN: {FNs}')
  F1 = TPs/ (TPs + 0.5 * (FPs + FNs) + 1e-16)

  if detailed: 
    output = (F1, F1s)
  else:
    output = F1
  return output

def compute_Jaccard(y_pred, y_gt, mask):
  '''
  Args
  -----
  y_pred: (n x 11)
  y_gt: (n x 11)
  mask: (n x 1)

  Returns
  -----
  Jaccard_accuracy: float
  '''

  n = mask.sum() + 1e-16
  y_pred = y_pred > 0.5
  intersect = ((y_pred==1)&(y_gt==1))*mask
  union = (((y_pred==1)|(y_gt==1)))*mask
  jaccards = intersect.sum(dim=-1) / (union.sum(dim=-1)+1e-16)
  jaccard = 1/n * jaccards.sum()

  return jaccard

  
def compute_corr(y_pred, y_gt, mask):
  '''
  Args
  -----
  y_pred: (n x 3)
  y_gt: (n x 3)
  mask: (n x 1)

  Returns
  -----
  Pearson Correlation Coefficient: arr
  '''
  rs = [0, 0, 0]

  y_pred_avg = (y_pred * mask).sum(dim=0) / (mask.sum() + 1e-16)
  y_gt_avg = (y_gt * mask).sum(dim=0) / (mask.sum() + 1e-16)

  for i in range(3):
    a = (y_pred[:, i] - y_pred_avg[i]) * mask.squeeze()
    b = (y_gt[:, i] - y_gt_avg[i]) * mask.squeeze()

    r = (a * b).sum() / (torch.sqrt((a * a).sum() * (b * b).sum()) + 1e-16)

    rs[i] = r.item()

  return rs


class MultiTaskMetric(object):
  def __init__(self):
    pass

  def __call__(self, y_pred, y_gt, task):
    ekman_count = task[:, 0].sum() + 1e-16
    vad_count = task[:, 1].sum() + 1e-16
    sem_count = task[:, 2].sum() + 1e-16

    metric = torch.zeros(3, dtype=torch.float, device=y_pred.device)

    # F1
    s1, f1 = 0, OUT_DIMS['ekman']
    s2, f2 = 0, Y_DIMS['ekman']
    pred = y_pred[:, s1:f1]
    gt = y_gt[:, s2:f2]
    mask = task[:, 0:1]
    F1 = compute_F1(pred, gt, mask)

    # Regression
    s1, f1 = s1+OUT_DIMS['ekman'], f1+OUT_DIMS['vad']
    s2, f2 = s2+Y_DIMS['ekman'], f2+Y_DIMS['vad']
    pred = y_pred[:, s1:f1]
    gt = y_gt[:, s2:f2]
    mask = task[:, 1:2]
    MSE = compute_corr(pred, gt, mask)
    
    # Jaccard
    s1, f1 = s1+OUT_DIMS['vad'], f1+OUT_DIMS['sem']
    s2, f2 = s2+Y_DIMS['vad'], f2+Y_DIMS['sem']
    pred = y_pred[:, s1:f1]
    gt = y_gt[:, s2:f2]
    mask = task[:, 2:]
    Jaccard = compute_Jaccard(pred, gt, mask)

    return F1, MSE, Jaccard



## 4 - Quantitative Evaluation over all test sets

The code in this section is similar to how we evaluate the network during training. Note that since our datasets are quite small, we can load everything into memory in one go to make computing the metrics much easier. 


In [44]:
metric = MultiTaskMetric()
net.eval()

dataloader = dataloaders['all']

n = len(dataloader.dataset)
y_preds = torch.empty(n, sum(OUT_DIMS.values()), dtype=torch.float) # n x 19
y_gts = torch.empty(n, sum(Y_DIMS.values()), dtype=torch.float) # n x 15
tasks = torch.empty(n, 3) # 3
i = 0
# if GPU is not on, this will take a while
for X, y, task in tqdm(dataloader): 
  b = len(X)
  y_pred = net(X, task)
  y_preds[i:i+b] = y_pred
  y_gts[i:i+b] = y
  tasks[i:i+b] = task
  i += b

f1, r, jaccard = metric(y_preds, y_gts, tasks)

  0%|          | 0/273 [00:00<?, ?it/s]

In [45]:
print('TEST SET PERFORMANCE SUMMARY')
print('---------------------------------')
print(f'F1 = {f1:.4f}')
print(f'Correlation = V: {r[0]:.4f}, A: {r[1]:.4f}, D: {r[2]:.4f}, r: {sum(r)/3:4f}')
print(f'Jaccard = {jaccard:.4f}')

TEST SET PERFORMANCE SUMMARY
---------------------------------
F1 = 0.7049
Correlation = V: 0.6496, A: 0.4344, D: 0.3706, r: 0.484844
Jaccard = 0.4502


## 5 - Qualitative Examples

There are three subsections here to illustrate some qualitative results. The first shows the predictions, the second shows raw outputs, and the third allows you to input some custom text to see what the network predicts

```
ekman_map = {'a-d': 'angry-disgust (ekman)', 
             'f|e': 'fear (ekman)', 
             'hap':'happy (ekman)',
             'sa|e':'sad (ekman)', 
             'sup|e': 'surprise (ekman)'}
sem_map   = {'ang': 'angry', 
             'antic': 'anticipation', 
             'disg': 'disgust',
             'f|s': 'fear (Sem)',
             'joy': 'joy', 
             'love': 'love', 
             'opt': 'optimism', 
             'pes': 'pessimism', 
             'sa|s': 'sad (sem)', 
             'sup|s': 'surprise (sem)', 
             'trust': 'trust'}
```

### Predictions 

You can run the following cells many times to see different samples

In [46]:
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

headings = ['sentence', 'task', 'label', 'pred']
rows = []

for name in ['ekman', 'vad', 'sem']:
  X, y, task = next(iter(dataloaders[name]))
  y_pred = net(X, task)
  # take first three datapoints
  for i in range(3):
    row = ["\n".join(wrap(X[i], 50)), name, '', '']
    if name == 'ekman':
      row[2] = EKMAN_EMOTIONS[int(y[i, 0])]
      pred = int(torch.argmax(y_pred[i, 0:3]))
      row[3] = EKMAN_EMOTIONS[pred]
    elif name == 'vad':
      row[2] = f'V: {y[i, 1]:.3f} \n A: {y[i, 2]:.3f} \n D: {y[i, 3]:.3f}'
      row[3] = f'V: {y_pred[i, 5]:.3f} \n A: {y_pred[i, 6]:.3f} \n D: {y_pred[i, 7]:.3f}'
    elif name == 'sem':
      pred = torch.round(y_pred[i, 8:])
      for j, p in enumerate(pred):
        if p == 1:
          row[3] += SEM_EMOTIONS[j] + ' '
      for j, p in enumerate(y[i, 4:]):
        if p == 1:
          row[2] += SEM_EMOTIONS[j] + ' '
    rows += [row]


df_pred = pd.DataFrame(rows, columns=headings)

# some settings to make df display better
pd.options.display.max_colwidth = 100 
df_pred['sentence'].str.wrap(12)

pretty_print(df_pred)

,sentence,task,label,pred
0,"The maiden looked for strawberries outside, but asshe found none, she went angrily home.",ekman,anger-disgust,anger-disgust
1,"Yes, I shall get to Egypt, for I feel so happy!",ekman,happy,happy
2,"Then they were all terrified, and the King tookthe bird, the fisherman and the three childrenback with him to the castle, and ordered theprison to be opened and brought his wife outagain.",ekman,fear,fear
3,It was those clues that started all of this.,vad,V: 3.000 A: 3.300 D: 3.100,V: 2.976 A: 2.991 D: 3.099
4,"No seventy-two virgins, but it would do.",vad,V: 3.000 A: 3.200 D: 3.000,V: 3.204 A: 3.025 D: 3.143
5,"On a sunny Sunday afternoon, the waiting list wasan hour long.",vad,V: 2.380 A: 2.880 D: 2.620,V: 2.965 A: 2.936 D: 3.002
6,#AmarnathTerrorAttack Muslims are killingeverywhere Syria Iraq Palestine Everyday beyondThey say that Islam is terrorism shame on you,sem,anger disgust fear,anger disgust fear
7,Every GIRL deserves a GUY who can make her SMILE:D,sem,joy optimism,joy optimism
8,@LadyScully I didn't. We all went out and gotpissed down the local instead. 😄,sem,anger disgust,


### Raw outputs

Note that the columns are shortened here to fit on the whole screen, the table is better viewed full screen by closing side bars.

In [47]:
ekman_short = ['a-d', 'f|e', 'hap', 'sa|e', 'sup|e']
sem_short = ['ang', 'antic', 'disg', 'f|s', 'joy', 'love', 'opt', 'pes', 'sa|s', 'sup|s', 'trust']
headings_raw = ['type', 'sentence', *ekman_short, *VAD_EMOTIONS, *sem_short]
rows = []

for name in ['ekman', 'vad', 'sem']:
  X, y, task = next(iter(dataloaders[name]))
  y_pred = net(X, ones_like(task))
  for i in range(3):
    row = ['pred', "\n".join(wrap(X[i], 60))]
    data = y_pred[i].detach().cpu()
    data[0:5] = torch.exp(data[0:5])
    row += [round(a.item(), 3) for a in data]
    
    rows += [row]

    gt = y[i].detach().cpu()
    ekman_onehot = [0, 0, 0, 0, 0]
    if task[i,0].item() == 1:
      ekman_onehot[int(gt[0])] = 1
    
    rows += [['label', '', *ekman_onehot, *[round(a.item(), 3) for a in gt[1:]]]]

rows += [[1] * len(row)]

df_raw = pd.DataFrame(rows, columns=headings_raw)

format_dict = {key:'{0:,.2f}' for key in headings_raw}
format_dict['type'] = '{:}'
format_dict['sentence'] = '{:}'

(df_raw.style
 .format(format_dict)
 .background_gradient(cmap="Purples", subset=ekman_short)
 .background_gradient(cmap='Blues', subset=sem_short)
 .hide([18]))

,type,sentence,a-d,f|e,hap,sa|e,sup|e,V,A,D,ang,antic,disg,f|s,joy,love,opt,pes,sa|s,sup|s,trust
0,pred,Then she bitterly grieved for the pride and folly which had brought her so low.,0.05,0.00,0.02,0.92,0.00,2.67,3.12,2.89,0.20,0.02,0.23,0.04,0.11,0.04,0.23,0.40,0.77,0.00,0.02
1,label,,0.00,0.00,0.00,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,pred,"Then the Princess would say, ""He is of no use!",0.72,0.15,0.06,0.00,0.06,2.64,3.18,3.05,0.06,0.16,0.64,0.02,0.14,0.09,0.27,0.01,0.15,0.12,0.00
3,label,,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
4,pred,"But I do not understand chattering and boasting; on the contrary, I despise it.""",0.15,0.00,0.60,0.24,0.00,2.97,3.23,3.13,0.32,0.07,0.30,0.01,0.26,0.03,0.42,0.18,0.38,0.01,0.04
5,label,,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
6,pred,"""My last piece was of four old Undersiders crouched around a trash fire.",0.89,0.00,0.09,0.00,0.03,3.08,2.96,3.10,0.34,0.15,0.42,0.04,0.40,0.07,0.33,0.08,0.19,0.04,0.06
7,label,,0.00,0.00,0.00,0.00,0.00,3.00,2.82,3.18,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
8,pred,She batted at my nose like a cat.,0.91,0.02,0.04,0.00,0.03,2.74,3.35,3.12,0.54,0.08,0.47,0.04,0.24,0.07,0.03,0.07,0.26,0.01,0.01
9,label,,0.00,0.00,0.00,0.00,0.00,3.00,3.10,3.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


### Qualitative Examples: Playground

Play around with your own text and see if you agree with the model! We've attached a few examples.

```
ekman_map = {'a-d': 'angry-disgust (ekman)', 
             'f|e': 'fear (ekman)', 
             'hap':'happy (ekman)',
             'sa|e':'sad (ekman)', 
             'sup|e': 'surprise (ekman)'}
sem_map   = {'ang': 'angry', 
             'antic': 'anticipation', 
             'disg': 'disgust',
             'f|s': 'fear (Sem)',
             'joy': 'joy', 
             'love': 'love', 
             'opt': 'optimism', 
             'pes': 'pessimism', 
             'sa|s': 'sad (sem)', 
             'sup|s': 'surprise (sem)', 
             'trust': 'trust'}
```


In [48]:
playground = ["The UCL NLP course was an amazing experience", 
              "A fruit forward cup with notes of orange and mango, this natural Mundo Novo is one of the best coffees we've tasted from Brazil",
              "I\'m gonna make him an offer he can\'t refuse.", 
              "You don't understand! I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.", 
              ]

tricky_sentences = ["I can't believe it's still not weekend yet",
                    "Do I seem happy to you?"]

# we asked chatgpt to give us some difficult examples (and it did):
gtp_sentences = ["The aroma of the freshly baked bread reminded him of his childhood, but also made him feel a sense of longing for a time that could never be regained.", 
                 "She listened to the sound of the rain tapping against the window, and felt a strange mixture of comfort and loneliness."]

playground += tricky_sentences + gtp_sentences

ekman_short = ['a-d', 'f|e', 'hap', 'sa|e', 'sup|e']
sem_short = ['ang', 'antic', 'disg', 'f|s', 'joy', 'love', 'opt', 'pes', 'sa|s', 'sup|s', 'trust']
headings_raw = ['sentence', *ekman_short, *VAD_EMOTIONS, *sem_short]
rows = []

y_pred = net(playground, torch.tensor([1, 1, 1], device=DEVICE).unsqueeze(0).repeat(len(playground), 1))
for i, s in enumerate(playground):
  row = ["\n".join(wrap(s, 60))]
  data = y_pred[i].detach().cpu()
  data[0:5] = torch.exp(data[0:5])
  row += [round(a.item(), 3) for a in data]
  
  rows += [row]

rows += [[1] * len(row)]

df_play = pd.DataFrame(rows, columns=headings_raw)

format_dict = {key:'{0:,.2f}' for key in headings_raw}
format_dict['sentence'] = '{:}'

(df_play.style
 .format(format_dict)
 .background_gradient(cmap="Purples", subset=ekman_short)
 .background_gradient(cmap='Blues', subset=sem_short)
 .hide([len(playground)]))

,sentence,a-d,f|e,hap,sa|e,sup|e,V,A,D,ang,antic,disg,f|s,joy,love,opt,pes,sa|s,sup|s,trust
0,The UCL NLP course was an amazing experience,0.00,0.00,0.97,0.00,0.03,3.66,3.21,3.22,0.01,0.44,0.01,0.06,0.93,0.21,0.83,0.03,0.03,0.14,0.13
1,"A fruit forward cup with notes of orange and mango, this natural Mundo Novo is one of the best coffees we've tasted from Brazil",0.01,0.00,0.98,0.00,0.01,3.44,3.15,3.17,0.03,0.24,0.03,0.01,0.94,0.24,0.68,0.02,0.04,0.12,0.09
2,I'm gonna make him an offer he can't refuse.,0.74,0.01,0.23,0.01,0.02,2.93,3.21,3.16,0.23,0.30,0.19,0.20,0.31,0.02,0.53,0.13,0.14,0.02,0.06
3,"You don't understand! I coulda had class. I coulda been a contender. I could've been somebody, instead of a bum, which is what I am.",1.00,0.00,0.00,0.00,0.00,2.57,3.20,3.17,0.94,0.06,0.88,0.04,0.02,0.00,0.11,0.18,0.26,0.01,0.02
4,I can't believe it's still not weekend yet,0.15,0.08,0.57,0.01,0.20,3.01,3.23,2.99,0.09,0.18,0.15,0.47,0.43,0.07,0.25,0.23,0.34,0.07,0.02
5,Do I seem happy to you?,0.63,0.09,0.08,0.02,0.18,2.93,2.99,3.03,0.15,0.18,0.27,0.19,0.11,0.01,0.17,0.34,0.44,0.05,0.02
6,"The aroma of the freshly baked bread reminded him of his childhood, but also made him feel a sense of longing for a time that could never be regained.",0.00,0.00,0.98,0.01,0.01,3.44,3.20,3.11,0.01,0.11,0.01,0.01,0.94,0.32,0.94,0.02,0.05,0.01,0.24
7,"She listened to the sound of the rain tapping against the window, and felt a strange mixture of comfort and loneliness.",0.03,0.24,0.06,0.65,0.03,2.91,2.99,2.86,0.01,0.10,0.04,0.41,0.18,0.04,0.32,0.34,0.62,0.01,0.03
